In [2]:
!which python

/srv/share5/fbai31/miniconda3/envs/axcell/bin/python


In [3]:
from collections import defaultdict
import random
import json

# Load unlabeled files

In [4]:
DATA_SOURCE = {
    'uspto_grant': '/srv/share5/fbai31/procedure/demo/raw/uspto_grant_1976_2000_merged',
    'uspto_app': '/srv/share5/fbai31/procedure/demo/raw/uspto_app_2001_2016_merged',
    'epo': '/srv/share5/fbai31/procedure/demo/raw/epo_merged',
    'pubmed': '/srv/share5/fbai31/procedure/demo/raw/pubmed_proceedures_merged',
}

In [5]:
def load_merged_chem_paragraphs(data_source, data_name='chemsyn'):
    doc_sens = defaultdict(list)
    merge_data_path = DATA_SOURCE[data_source]
    print("Load merged files from:", merge_data_path)

    with open(merge_data_path, 'r') as fh:
        for line_idx, line in enumerate(fh):

            if line_idx and line_idx % 500000 == 0:
                print(line_idx, len(doc_sens))

            if data_name in ['chemsyn', 'chemu']:

                if len(line.strip().split('&&&&&&')) != 2:
                    print(line)
                    continue

                doc_id, sen_text = line.strip().split('&&&&&&')

            elif data_name == 'pubmed':

                if 'https://www' in line:
                    continue

                if len(line.strip().split('\t')) < 2:
                    continue

                if len(line.strip().split('\t')) > 2:
                    # print(line)
                    continue

                doc_id, sen_text = line.strip().split('\t')[:2]

            doc_sens[doc_id].append(sen_text)
    print("# files:", len(doc_sens))
    
    return doc_sens

In [6]:
doc_sens = load_merged_chem_paragraphs('uspto_app')

Load merged files from: /srv/share5/fbai31/procedure/demo/raw/uspto_app_2001_2016_merged
500000 87037
1000000 175003
1500000 261299
2000000 348183
2500000 434575
3000000 521467
3500000 607812
4000000 693353
4500000 780369
5000000 865912
5500000 951264
6000000 1036757
6500000 1121858
7000000 1207659
7500000 1293230
8000000 1377044
8500000 1460845
9000000 1544518
9500000 1627674
10000000 1710660
10500000 1794055
11000000 1875879
# files: 1939253


In [7]:
doc_names = list(doc_sens.keys())
random.seed(1234)
random.shuffle(doc_names)
doc_names[:10]

['I20150219_55_US20150048284A1',
 '20030327_169_US20030060453A1',
 'I20160519_4140_US20160139133A1',
 '20040219_34_US20040033896A1',
 'I20141002_3204_US20140296208A1',
 '20020131_481_US20020013328A1',
 'I20111208_3063_US20110301203A1',
 'I20100916b_9_US20100233086A1',
 'I20131017_3285_US20130274475A1',
 'I20160714_3665_US20160200686A1']

In [10]:
doc_sens['I20160519_4140_US20160139133A1']

['NoHeadingText',
 'To benzyl 2-(benzyloxy)-5-(2-hydroxyacetyl)benzoate (80 mg, 0.21 mmol) in anhydrous THF (2 mL) was added 1-((4-(tert-butyl)phenoxy)carbonyl)-3-methyl-1H-imidazol-3-ium tetrafluoroboroate (55 mg, 0.21 mmol) and the mixture was stirred at room temperature overnight.',
 'TLC showed that the reaction was incomplete and the mixture was heated at reflux for 5 h.',
 'The impure material was dry-loaded onto silica and was purified using a Biotage Isolera automated chromatography system under normal phase conditions (silica column, gradient of 10→100% ethyl acetate in petrol) with detection at 254 nm to give benzyl 2-(benzyloxy)-5-(2-(((4-(tert-butyl)phenoxy)carbonyl)oxy)acetyl)benzoate (88 mg, 80%), as a colorless oil.']

In [11]:
doc_sens_selected = {}

for doc_name in doc_names:
    
    if doc_sens[doc_name][0] != 'NoHeadingText' and len(doc_sens[doc_name]) > 5 and '1H NMR' not in ' '.join(doc_sens[doc_name]):
        
        if len(doc_sens_selected) >= 5000:
            break
        
        doc_sens_selected[doc_name] = doc_sens[doc_name]
    
#     break

In [12]:
len(doc_sens_selected)

5000

In [13]:
!pwd

/coc/pskynet5/fbai31/in_context_anaphora/fan/notebooks


In [11]:
!ls ../../data

ChemuRef_v2  ChemuRef_v3  unlabeled


# Dump unlabeled data

In [14]:
unlabeled_filepath = '/srv/share5/fbai31/in_context_anaphora/data/unlabeled/uspto_5000_paras.json'

In [15]:
with open(unlabeled_filepath, "w") as f:
    json.dump(doc_sens_selected, f, indent=4)